## Teoria

O Algoritmo Apriori é um algoritmo clássico de mineração de dados usado para encontrar **conjuntos de itens frequentes** e **regras de associação** em grandes bancos de dados transacionais. É amplamente utilizado em análise de cestas de compras para entender quais itens são frequentemente comprados juntos.

<br>

### Conceitos Chave:

*   **Conjunto de Itens (Itemset):** Uma coleção de um ou mais itens (ex: {Pão, Leite}).
*   **Suporte (Support):** A frequência com que um conjunto de itens aparece no conjunto de dados. É a proporção de transações que contêm o conjunto de itens.
    * ```Suporte(X) = (Número de Transações Contendo X) / (Número Total de Transações)```.
*   **Regra de Associação:** Uma implicação da forma X -> Y, onde X e Y são conjuntos de itens disjuntos. Ela significa que se X ocorre, então Y provavelmente também ocorrerá. (ex: {Cerveja} -> {Fraldas}).
*   **Confiança (Confidence):** Mede a probabilidade de Y ser comprado quando X já foi comprado.
    * ```Confiança(X -> Y) = Suporte(X U Y) / Suporte(X)```.

*   **Lift (Elevador):** Mede o quanto a ocorrência de X e Y juntos é mais (ou menos) provável do que se eles fossem independentes. Um `lift` > 1 indica uma associação positiva, < 1 uma associação negativa, e = 1 indica independência.
    * ```Lift(X -> Y) = Confiança(X -> Y) / Suporte(Y)```.

<br>

### Como funciona o Apriori?

O algoritmo Apriori segue um processo iterativo para encontrar conjuntos de itens frequentes:

1.  **Geração de Candidatos:** Começa encontrando a frequência de cada item individualmente (itemsets de tamanho 1). Aqueles que atendem a um `suporte mínimo` são considerados frequentes.

2.  **Poda (Pruning):** Usa a propriedade *Apriori* (se um itemset é frequente, então todos os seus subconjuntos também devem ser frequentes) para 'podar' os candidatos. Se um subconjunto de um itemset candidato não for frequente, então o itemset candidato não pode ser frequente e é descartado.

3.  **Iteração:** Repete os passos de geração de candidatos e poda para itemsets de tamanho maior (2, 3, etc.) até que nenhum novo conjunto de itens frequente possa ser encontrado.

4.  **Geração de Regras:** Uma vez que todos os conjuntos de itens frequentes são encontrados, as regras de associação são geradas a partir deles, avaliando sua `confiança` e `lift`.

### Exemplo
![priori](https://github.com/AdrianoR85/Estatistica-para-Machine-Learning/blob/main/assets/priori.png?raw=true)

## Prática

In [ ]:
!pip install apyori

In [32]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from apyori import apriori


In [61]:
# 1. Transforming data into categorical values

dataset = pd.read_csv('/content/drive/MyDrive/Estatística para Ciência de Dados e Machine Learning/Bases de dados/census.csv')
dataset['age'] = pd.cut(dataset['age'], bins=[0,17,25,40,60,90], labels=['Faixa1', 'Faixa2', 'Faixa3', 'Faixa4', 'Faixa5'])
dataset.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,Faixa3,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Faixa4,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Faixa3,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Faixa4,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Faixa3,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [62]:
# 2. Select the desired columns for analysis.

dataset_apriori = dataset[['age', 'workclass', 'education','marital-status','relationship','occupation','sex', 'native-country','income']]
dataset_apriori.head()

,age,workclass,education,marital-status,relationship,occupation,sex,native-country,income
0,Faixa3,State-gov,Bachelors,Never-married,Not-in-family,Adm-clerical,Male,United-States,<=50K
1,Faixa4,Self-emp-not-inc,Bachelors,Married-civ-spouse,Husband,Exec-managerial,Male,United-States,<=50K
2,Faixa3,Private,HS-grad,Divorced,Not-in-family,Handlers-cleaners,Male,United-States,<=50K
3,Faixa4,Private,11th,Married-civ-spouse,Husband,Handlers-cleaners,Male,United-States,<=50K
4,Faixa3,Private,Bachelors,Married-civ-spouse,Wife,Prof-specialty,Female,Cuba,<=50K


In [18]:
dataset.shape

(32561, 15)

In [63]:
# 3. Take a sample from the dataset
dataset_apriori = dataset_apriori.sample(n=1000)

In [64]:
# 4. Transforme the dataset into a list
transactions = []
for i in range(dataset_apriori.shape[0]):
  transactions.append([str(dataset_apriori.values[i,j]) for j in range(dataset_apriori.shape[1])])

transactions[:2]

[['Faixa3',
  ' Private',
  ' Bachelors',
  ' Never-married',
  ' Not-in-family',
  ' Exec-managerial',
  ' Male',
  ' United-States',
  ' <=50K'],
 ['Faixa4',
  ' Private',
  ' HS-grad',
  ' Divorced',
  ' Own-child',
  ' Other-service',
  ' Female',
  ' South',
  ' <=50K']]

In [65]:
# 5. Create the rules
rules = apriori(transactions, min_support = 0.3, min_confidence = 0.6)
result = list(rules)

In [66]:
len(result)

30

In [67]:
for r in result:
  print(r)

RelationRecord(items=frozenset({' <=50K'}), support=0.76, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({' <=50K'}), confidence=0.76, lift=1.0)])
RelationRecord(items=frozenset({' Male'}), support=0.666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({' Male'}), confidence=0.666, lift=1.0)])
RelationRecord(items=frozenset({' Private'}), support=0.675, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({' Private'}), confidence=0.675, lift=1.0)])
RelationRecord(items=frozenset({' United-States'}), support=0.897, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({' United-States'}), confidence=0.897, lift=1.0)])
RelationRecord(items=frozenset({' Female', ' <=50K'}), support=0.306, ordered_statistics=[OrderedStatistic(items_base=frozenset({' Female'}), items_add=frozenset({' <=50K'}), confidence=0.9161676646706586, lift=1.205483769303498)])
RelationRecord(items

In [91]:
for relation_record in result[:8]:
  for ordered_statistic in relation_record.ordered_statistics:
    antecedent = ', '.join(list(ordered_statistic.items_base))
    consequent = ', '.join(list(ordered_statistic.items_add))
    print(f'Rule: {antecedent} -> {consequent}')
    print(f'Support: {relation_record.support}')
    print(f'Confidence: {ordered_statistic.confidence}')
    print(f'Lift: {ordered_statistic.lift}\n')

Rule:  ->  <=50K
Support: 0.76
Confidence: 0.76
Lift: 1.0

Rule:  ->  Male
Support: 0.666
Confidence: 0.666
Lift: 1.0

Rule:  ->  Private
Support: 0.675
Confidence: 0.675
Lift: 1.0

Rule:  ->  United-States
Support: 0.897
Confidence: 0.897
Lift: 1.0

Rule:  Female ->  <=50K
Support: 0.306
Confidence: 0.9161676646706586
Lift: 1.205483769303498

Rule:  Male ->  <=50K
Support: 0.454
Confidence: 0.6816816816816816
Lift: 0.8969495811601074

Rule:  Never-married ->  <=50K
Support: 0.318
Confidence: 0.9464285714285714
Lift: 1.2453007518796992

Rule:  <=50K ->  Private
Support: 0.524
Confidence: 0.6894736842105263
Lift: 1.0214424951267056

Rule:  Private ->  <=50K
Support: 0.524
Confidence: 0.7762962962962963
Lift: 1.0214424951267056



### Entendendo as Regras de Associação Geradas

As regras de associação são apresentadas no formato `Antecedente -> Consequente`, seguidas de suas métricas: Suporte, Confiança e Lift. Vamos entender o que cada parte significa e como interpretar os resultados.

**Componentes de uma Regra:**

*   **Antecedente (`Left-Hand Side` ou `items_base`)**: O conjunto de itens 'SE'. São os itens que aparecem juntos antes que o consequente ocorra.
*   **Consequente (`Right-Hand Side` ou `items_add`)**: O conjunto de itens 'ENTÃO'. São os itens que tendem a aparecer quando o antecedente está presente.

**Métricas de Avaliação:**

*   **Suporte (Support)**: Indica a frequência com que o conjunto completo de itens (Antecedente + Consequente) aparece no dataset. Um suporte de 0.306 significa que 30.6% das transações contêm ambos os conjuntos de itens.

*   **Confiança (Confidence)**: Mede a probabilidade de o Consequente ocorrer **dado que** o Antecedente já ocorreu. Uma confiança de 0.916 significa que, em 91.6% das vezes que o Antecedente aparece, o Consequente também aparece.

*   **Lift (Elevador)**: Avalia o quanto a ocorrência conjunta do Antecedente e Consequente é mais (ou menos) provável do que se eles fossem independentes.
    *   `Lift > 1`: Associação positiva. Os itens tendem a aparecer juntos mais do que por acaso. Quanto maior o Lift, mais forte a associação.
    *   `Lift = 1`: Independência. A ocorrência de um não afeta a do outro.
    *   `Lift < 1`: Associação negativa. Os itens tendem a não aparecer juntos.

---

### Exemplos de Interpretação das Regras

Vamos usar os exemplos fornecidos para ilustrar a interpretação:

1.  **Regra com Antecedente Vazio:**
    *   **`Rule:  -> United-States`**
    *   **`Support: 0.897`**
    *   **`Confidence: 0.897`**
    *   **`Lift: 1.0`**

    *   **Explicação:** Quando o Antecedente está vazio (` `), a regra não indica uma relação condicional, mas sim a **frequência geral do Consequente** (`United-States`) no dataset.
        *   **Suporte (0.897):** Significa que 89.7% das entradas no seu dataset têm a característica 'native-country' como 'United-States'.
        *   **Confiança (0.897):** Neste caso, a confiança é igual ao suporte, pois não há condição para 'anteceder'. É a probabilidade de um registro aleatório ser dos 'United-States'.
        *   **Lift (1.0):** Um Lift de 1.0 é esperado para regras com antecedente vazio, indicando que a ocorrência do consequente é independente de qualquer outro fator (pois não há antecedente).

2.  **Regra com Antecedente Não Vazio:**
    *   **`Rule: Female -> <=50K`**
    *   **`Support: 0.306`**
    *   **`Confidence: 0.9161676646706586`**
    *   **`Lift: 1.205483769303498`**

    *   **Explicação:** Esta é uma regra de associação condicional típica:
        *   **Antecedente (`Female`):** Se a pessoa é 'Female' (mulher).
        *   **Consequente (`<=50K`):** Então a renda é 'less than or equal to 50K'.
        *   **Suporte (0.306):** 30.6% de todas as entradas no dataset correspondem a mulheres com renda de `<=50K`.
        *   **Confiança (0.916):** Para 91.6% das mulheres no dataset, sua renda é `<=50K`. Esta é uma alta probabilidade condicional.
        *   **Lift (1.205):** Este valor é maior que 1, indicando uma **associação positiva**. Significa que as mulheres têm 1.205 vezes mais probabilidade de ter uma renda de `<=50K` do que a população geral. Ou seja, a condição 'Female' **aumenta a probabilidade** de a renda ser `<=50K` em 20.5% em comparação com a probabilidade base (sem considerar o gênero).